In [0]:
#@title Default title text

import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import mlflow
import mlflow.pytorch

#PARAMS


In [0]:
class Params(object):
    def __init__(self,batch_size,epochs,seed,log_interval):
        self.batch_size=batch_size
        self.epochs=epochs
        self.seed=seed
        self.log_interval=log_interval
args=Params(256,4,0,20)

#DATASET


In [0]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

trainset=datasets.MNIST('/Desktop/data',train=True,download=True,transform=transform)
testset=datasets.MNIST('/Desktop/data',train=False,download=True,transform=transform)

trainloader=torch.utils.data.DataLoader(trainset,shuffle=True,batch_size=args.batch_size)
testloader=torch.utils.data.DataLoader(testset,shuffle=False,batch_size=args.batch_size)

#MODEL


In [0]:
class Model(nn.Module):
    def __init__(self,nH=32):
        super(Model,self).__init__()
        self.classifier=nn.Sequential(
            nn.Linear(784,nH),
            nn.ReLU(),
            nn.Linear(nH,10)
        )
    
    def forward(self,x):
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        return x
    

In [0]:
def train(epoch):
    model.train()
    
    for batch_id,data in enumerate(trainloader):
        inputs,labels=data
        opt.zero_grad()
        outputs=model(inputs)
        loss=loss_fn(outputs,labels)
        loss.backward()
        opt.step()
        
        if batch_id % args.log_interval ==0:
            pos=epoch* len(trainloader) + batch_id
            mlflow.log_metric('train_loss',loss.data.item()/len(inputs)*1000)
            
            print('Train Epoch: {} [{}/{} ( {:0.2f} % )] \t Loss : {:.3f}'.format(
            epoch,batch_id * len(inputs),len(trainloader.dataset),100.*batch_id/len(trainloader),loss.data.item()))

In [0]:
def test(epoch):
    model.eval()
    test_loss=0
    correct=0
    confusion_matrix=np.zeros([10,10])
    
    with torch.no_grad():
        for inputs,labels in testloader:
            outputs=model(inputs)
            test_loss+=loss_fn(outputs,labels).data.item()
            pred=outputs.data.max(1)[1]
            correct +=pred.eq(labels.data).sum().item()
            
            for x,y, in zip(pred.numpy(),labels.numpy()):
                confusion_matrix[x][y] +=1
        test_loss /=len(testloader.dataset)
        test_accuracy=100.0 * correct/len(testloader.dataset)
        
        pos=(epoch+1)*len(trainloader)
        mlflow.log_metric('test_loss',test_loss*1000)
        mlflow.log_metric('test accuracy',test_accuracy)
        
        print("\n Test Set: Average Loss: {:.4f}, Accuracy : {}/{} ({:.0f}%\n".format(test_loss,correct,len(testloader.dataset),test_accuracy))
        
        
        if epoch==args.epochs:
            classes=np.arange(10)
            fig,ax=plt.subplots()
            im=ax.imshow(confusion_matrix,interpolation='nearest',cmap=plt.cm.Blues)
            ax.figure.colorbar(im,ax=ax)
            ax.set(xticks=np.arange(confusion_matrix.shape[1]),
                                    yticks=np.arange(confusion_matrix.shape[0]),
                                                    xticklabels=classes,yticklabels=classes,
                                                    ylabel='True Label',
                                                    xlabel='Predicted Label',
                                                    title="Epoch %d"% epoch)
            thres = confusion_matrix.max() /2
            for i in range(confusion_matrix.shape[0]):
                for j in range(confusion_matrix.shape[1]):
                   ax.text(j,i,int(confusion_matrix[i,j]),ha="center",va="center",color="white" if confusion_matrix[i,j] > thres else "black")
            fig.tight_layout()
            image_path="%s.png" % (expt_id)
            plt.savefig(image_path)
            mlflow.log_artifact(image_path)

#MLFLOW Hyper Parameter Tuning



In [0]:
#expt_id=mlflow.create_experiment('Fixed nH')
mlflow.set_experiment('Fixed NH')
for lr in [0.01]:
    for momemtum in [0.9]:
        for hidden_nodes in [16,32,48]:
            
            expt_id='%d_%d_%d' % (int(lr*100),int(momemtum*100),hidden_nodes)
            
            print("\n LR= %.2f, Momemtum =%.2f , Hidden nodes = %.2f" % (lr,momemtum,hidden_nodes))
            
            model=Model(hidden_nodes)
            loss_fn=nn.CrossEntropyLoss()
            opt=optim.SGD(model.parameters(),lr=lr,momentum=momemtum)
            
            with mlflow.start_run() as run:
                for key,value in vars(args).items():
                    mlflow.log_param(key,value)
                mlflow.log_param('lr',lr)
                mlflow.log_param('momemtum',momemtum)
                mlflow.log_param('Hidden_nodes',hidden_nodes)
                
                for epoch in range(1,args.epochs+1):
                    train(epoch)
                    test(epoch)
                    mlflow.pytorch.log_model(model,"models")
                


 LR= 0.01, Momemtum =0.90 , Hidden nodes = 16.00
Train Epoch: 1 [0/60000 ( 0.00 % )] 	 Loss : 2.374
Train Epoch: 1 [5120/60000 ( 8.51 % )] 	 Loss : 1.445
Train Epoch: 1 [10240/60000 ( 17.02 % )] 	 Loss : 0.500
Train Epoch: 1 [15360/60000 ( 25.53 % )] 	 Loss : 0.384
Train Epoch: 1 [20480/60000 ( 34.04 % )] 	 Loss : 0.406
Train Epoch: 1 [25600/60000 ( 42.55 % )] 	 Loss : 0.411
Train Epoch: 1 [30720/60000 ( 51.06 % )] 	 Loss : 0.288
Train Epoch: 1 [35840/60000 ( 59.57 % )] 	 Loss : 0.388
Train Epoch: 1 [40960/60000 ( 68.09 % )] 	 Loss : 0.334
Train Epoch: 1 [46080/60000 ( 76.60 % )] 	 Loss : 0.334
Train Epoch: 1 [51200/60000 ( 85.11 % )] 	 Loss : 0.271
Train Epoch: 1 [56320/60000 ( 93.62 % )] 	 Loss : 0.296

 Test Set: Average Loss: 0.0011, Accuracy : 9218/10000 (92%



C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 2 [0/60000 ( 0.00 % )] 	 Loss : 0.288
Train Epoch: 2 [5120/60000 ( 8.51 % )] 	 Loss : 0.314
Train Epoch: 2 [10240/60000 ( 17.02 % )] 	 Loss : 0.292
Train Epoch: 2 [15360/60000 ( 25.53 % )] 	 Loss : 0.202
Train Epoch: 2 [20480/60000 ( 34.04 % )] 	 Loss : 0.234
Train Epoch: 2 [25600/60000 ( 42.55 % )] 	 Loss : 0.199
Train Epoch: 2 [30720/60000 ( 51.06 % )] 	 Loss : 0.177
Train Epoch: 2 [35840/60000 ( 59.57 % )] 	 Loss : 0.221
Train Epoch: 2 [40960/60000 ( 68.09 % )] 	 Loss : 0.197
Train Epoch: 2 [46080/60000 ( 76.60 % )] 	 Loss : 0.186
Train Epoch: 2 [51200/60000 ( 85.11 % )] 	 Loss : 0.322
Train Epoch: 2 [56320/60000 ( 93.62 % )] 	 Loss : 0.200

 Test Set: Average Loss: 0.0009, Accuracy : 9353/10000 (94%



C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 3 [0/60000 ( 0.00 % )] 	 Loss : 0.195
Train Epoch: 3 [5120/60000 ( 8.51 % )] 	 Loss : 0.186
Train Epoch: 3 [10240/60000 ( 17.02 % )] 	 Loss : 0.263
Train Epoch: 3 [15360/60000 ( 25.53 % )] 	 Loss : 0.270
Train Epoch: 3 [20480/60000 ( 34.04 % )] 	 Loss : 0.125
Train Epoch: 3 [25600/60000 ( 42.55 % )] 	 Loss : 0.274
Train Epoch: 3 [30720/60000 ( 51.06 % )] 	 Loss : 0.215
Train Epoch: 3 [35840/60000 ( 59.57 % )] 	 Loss : 0.232
Train Epoch: 3 [40960/60000 ( 68.09 % )] 	 Loss : 0.255
Train Epoch: 3 [46080/60000 ( 76.60 % )] 	 Loss : 0.195
Train Epoch: 3 [51200/60000 ( 85.11 % )] 	 Loss : 0.231
Train Epoch: 3 [56320/60000 ( 93.62 % )] 	 Loss : 0.347

 Test Set: Average Loss: 0.0008, Accuracy : 9406/10000 (94%



...

Train Epoch: 2 [0/60000 ( 0.00 % )] 	 Loss : 0.371
Train Epoch: 2 [5120/60000 ( 8.51 % )] 	 Loss : 0.200
Train Epoch: 2 [10240/60000 ( 17.02 % )] 	 Loss : 0.272
Train Epoch: 2 [15360/60000 ( 25.53 % )] 	 Loss : 0.266
Train Epoch: 2 [20480/60000 ( 34.04 % )] 	 Loss : 0.287
Train Epoch: 2 [25600/60000 ( 42.55 % )] 	 Loss : 0.264
Train Epoch: 2 [30720/60000 ( 51.06 % )] 	 Loss : 0.292
Train Epoch: 2 [35840/60000 ( 59.57 % )] 	 Loss : 0.173
Train Epoch: 2 [40960/60000 ( 68.09 % )] 	 Loss : 0.226
Train Epoch: 2 [46080/60000 ( 76.60 % )] 	 Loss : 0.220
Train Epoch: 2 [51200/60000 ( 85.11 % )] 	 Loss : 0.246
Train Epoch: 2 [56320/60000 ( 93.62 % )] 	 Loss : 0.194

 Test Set: Average Loss: 0.0008, Accuracy : 9392/10000 (94%



C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 3 [0/60000 ( 0.00 % )] 	 Loss : 0.307
Train Epoch: 3 [5120/60000 ( 8.51 % )] 	 Loss : 0.206
Train Epoch: 3 [10240/60000 ( 17.02 % )] 	 Loss : 0.206
Train Epoch: 3 [15360/60000 ( 25.53 % )] 	 Loss : 0.264
Train Epoch: 3 [20480/60000 ( 34.04 % )] 	 Loss : 0.199
Train Epoch: 3 [25600/60000 ( 42.55 % )] 	 Loss : 0.233
Train Epoch: 3 [30720/60000 ( 51.06 % )] 	 Loss : 0.175
Train Epoch: 3 [35840/60000 ( 59.57 % )] 	 Loss : 0.158
Train Epoch: 3 [40960/60000 ( 68.09 % )] 	 Loss : 0.211
Train Epoch: 3 [46080/60000 ( 76.60 % )] 	 Loss : 0.161
Train Epoch: 3 [51200/60000 ( 85.11 % )] 	 Loss : 0.164
Train Epoch: 3 [56320/60000 ( 93.62 % )] 	 Loss : 0.185

 Test Set: Average Loss: 0.0007, Accuracy : 9460/10000 (95%



C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 4 [0/60000 ( 0.00 % )] 	 Loss : 0.129
Train Epoch: 4 [5120/60000 ( 8.51 % )] 	 Loss : 0.203
Train Epoch: 4 [10240/60000 ( 17.02 % )] 	 Loss : 0.162
Train Epoch: 4 [15360/60000 ( 25.53 % )] 	 Loss : 0.186
Train Epoch: 4 [20480/60000 ( 34.04 % )] 	 Loss : 0.210
Train Epoch: 4 [25600/60000 ( 42.55 % )] 	 Loss : 0.142
Train Epoch: 4 [30720/60000 ( 51.06 % )] 	 Loss : 0.294
Train Epoch: 4 [35840/60000 ( 59.57 % )] 	 Loss : 0.202
Train Epoch: 4 [40960/60000 ( 68.09 % )] 	 Loss : 0.187
Train Epoch: 4 [46080/60000 ( 76.60 % )] 	 Loss : 0.130
